In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
sys.path.append(r'C:\Temp\SoundOfAI\rg_text_to_sound\tts_pipeline\src')
from match_word_to_words import word_to_words_matcher
target_word_pairs = [('Bright', 'Dark'), ('Full', 'Hollow'),( 'Smooth', 'Rough'), ('Warm', 'Metallic'), ('Clear', 'Muddy'), ('Thin', 'Thick'), ('Pure', 'Noisy'), ('Rich', 'Sparse'), ('Soft', 'Hard')]

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('text_to_qualities.csv')
colnames = df.columns
display(df.head(2))
df.shape

,bright_vs_dark,full_vs_hollow,smooth_vs_rough,warm_vs_metallic,clear_vs_muddy,thin_vs_thick,pure_vs_noisy,rich_vs_sparse,soft_vs_hard,description
0,63,25,60,75,11,66,93,19,50,sitar-like
1,68,29,71,22,31,76,35,29,50,Full


(1303, 10)

In [125]:
wordlist = []
rowlist =[]
for r,w in generate_training_examples(df):
    wordlist.append(w)
    rowlist.append(r)

In [127]:
unique_word_list = np.unique(wordlist).tolist()
len(wordlist),len(unique_word_list)

(2667, 670)

# word2word matcher

In [3]:
def generate_training_examples(df):
    for irow,row in df.iterrows():
        if ', ' in row.description:
            splitlist = row.description.split(', ')
        else:
            splitlist=[row.description]
        for word in splitlist:
            yield row,word

In [7]:
target_word_list = [tup[0] for tup in target_word_pairs]+[tup[1] for tup in target_word_pairs]

In [27]:
target_word_list;

In [60]:
w2wm = word_to_words_matcher()

In [61]:
import ipdb
print('x')
w2wm.build(target_word_list)

x


In [62]:
w2wm.clusterer.cluster_centers_.shape

(18, 300)

In [63]:
w2wm.match_word_to_words('greek')

'Full'

In [68]:
w2wm.predict(['greek','sluggish'])

['Full', 'Rough']

In [59]:
%debug

> c:\temp\soundofai\rg_text_to_sound\playground\beat_toedtli\word2word_matching\match_word_to_words.py(70)predict()
     68         vector_array = self.get_vector_array(words)
     69         clusterind = self.clusterer.predict(vector_array)
---> 70         return self.target_tokens[clusterind].tolist()
     71 
     72     def dispose(self):



ipdb>  clusterind


array([ 1, 11])


ipdb>  self.target_tokens_array[clusterind].tolist()


['Full', 'Rough']


ipdb>  q


In [73]:
w2wm.predict_index(wordlist)

array([ 1,  1, 15, ..., 17, 16, 17])

In [69]:
def max_occurrence_prediction(wordlist,w2wm,variant=1):
    """
    predict that word that occurs most often in a wordlist
    """
    #set_trace()
    if variant==2:
        yhat = []
        for word in wordlist:
            yhat_elems = w2wm.predict([word])
            yhat_elem = max(set(yhat_elems), key = l.count)
            yhat.append(yhat_elem)
    elif variant==1:
        yhat = [max(set(w2wm.predict([word])), key = l.count) for word in wordlist]

    return yhat
yhat1 = np.array(max_occurrence_prediction(wordlist,w2wm,variant=1))
yhat2 = np.array(max_occurrence_prediction(wordlist,w2wm,variant=2))
yhat1==yhat2

array([ True,  True,  True, ...,  True,  True,  True])

In [78]:
pd.Series(yhat1).unique()

array(['Full', 'Noisy', 'Sparse', 'Rough', 'Clear', 'Thin', 'Hollow',
       'Soft', 'Hard', 'Dark', 'Bright', 'Warm', 'Pure', 'Muddy',
       'Metallic', 'Smooth', 'Thick', 'Rich'], dtype=object)

0         sitar-like
1               Full
2            wailing
3              greek
4            zooming
            ...     
2662         country
2663       distorted
2664        bouncing
2665           zoomy
2666    blues guitar
Length: 2667, dtype: object

In [ ]:
for col in df_pairs:
    

In [113]:
for irow,(col,word,word_pair) in enumerate(zip(df_pairs.columns,ser_words,target_word_pairs)):
    curr_col = df_pairs.iloc[irow,:][col]
    #w2wm = word_to_words_matcher()
    
    print(word_pair,col)
word,col,curr_col,word_pair

('Bright', 'Dark') bright_vs_dark
('Full', 'Hollow') full_vs_hollow
('Smooth', 'Rough') smooth_vs_rough
('Warm', 'Metallic') warm_vs_metallic
('Clear', 'Muddy') clear_vs_muddy
('Thin', 'Thick') thin_vs_thick
('Pure', 'Noisy') pure_vs_noisy
('Rich', 'Sparse') rich_vs_sparse
('Soft', 'Hard') soft_vs_hard


('distorted', 'soft_vs_hard', 50, ('Soft', 'Hard'))

In [4]:
ser_words = wordlist = pd.Series(wordlist)
df_pairs = pd.DataFrame([currRow[currRow.index[:-1]] for currRow in rowlist])

wordpair_matcher_dict=dict()
for col,word_pair in zip(df_pairs.columns,target_word_pairs):
    w2wm = word_to_words_matcher()
    w2wm.build(list(word_pair))
    wordpair_matcher_dict[col]=w2wm
col,word_pair

NameError: name 'wordlist' is not defined

# word pair estimator

In [95]:
from match_word_to_words import word_to_wordpair_estimator
df_pairs = pd.DataFrame([currRow[currRow.index[:-1]] for currRow in rowlist])

w2wpe = word_to_wordpair_estimator()
w2wpe.build(df_pairs.columns.tolist(),target_word_pairs)

In [100]:
dummy = 'dummy'
print(f'{dummy:<10}',f'{0.000:1.3f}', 0,0,'closer to ',dummy,'than',dummy)
(w2wpe.match_word_to_wordpair('bright','bright_vs_dark'),
 w2wpe.match_word_to_wordpair('shiny','bright_vs_dark'),
 w2wpe.match_word_to_wordpair('gloomy','bright_vs_dark'),
 w2wpe.match_word_to_wordpair('day','bright_vs_dark'),
 w2wpe.match_word_to_wordpair('night','bright_vs_dark'),
 w2wpe.match_word_to_wordpair('dark','bright_vs_dark'),
 w2wpe.match_word_to_wordpair('useless','bright_vs_dark')
);

dummy      0.000 0 0 closer to  dummy than dummy
bright     0.000 0 1 closer to  Bright than Dark
shiny      0.470 0 1 closer to  Bright than Dark
gloomy     0.523 1 0 closer to  Dark than Bright
day        0.497 0 1 closer to  Bright than Dark
night      0.505 1 0 closer to  Dark than Bright
dark       1.000 1 0 closer to  Dark than Bright
useless    0.499 0 1 closer to  Bright than Dark


In [134]:
a=[1,2]
a.append(3)
a

[1, 2, 3]

In [203]:
def create_threshold_plot(wordpairname,w2wpe,thlist=None):
    if thlist is None:
        thlist = [0,0.2,*np.linspace(0.4,0.5,20).tolist()]
    countlist = []
    for threshold in thlist:
        count = 0
        for word in unique_word_list:
            d = w2wpe.match_word_to_wordpair(word,wordpairname)
            proximity,sliderval,closest,fartherst = d['proximity'],d['slider value'],d['closest word'],d['other word']
            if proximity < threshold:
                #print(f'{word:<20}',f'{proximity:1.3f}',f'{sliderval:1.3f}',closest,fartherst)
                count +=1
        countlist.append(count)
    return countlist

In [195]:
d

{'proximity': 0.0,
 'slider value': 0.0,
 'closest dist': 0.0,
 'other dist': 6.263573686346848,
 'closest word': 'Soft',
 'other word': 'Hard'}

In [196]:
for wpname in w2wpe.wordpair_names_dict:
    twp = w2wpe.wordpair_names_dict[wpname]
    d = w2wpe.match_word_to_wordpair(twp[0],wpname)
    d2 = w2wpe.match_word_to_wordpair(twp[1],wpname)
    #print(d['slider value'],d2['slider value'],d['other dist'])
    print(f'{wpname:<20}',f"{d['other dist']:1.3f}")

bright_vs_dark       5.497
full_vs_hollow       7.630
smooth_vs_rough      6.122
warm_vs_metallic     8.490
clear_vs_muddy       7.037
thin_vs_thick        4.594
pure_vs_noisy        8.466
rich_vs_sparse       7.708
soft_vs_hard         6.264


In [ ]:
lang_model= 'en_core_web_sm'
w2wpe = word_to_wordpair_estimator()
w2wpe.build(df_pairs.columns.tolist(),target_word_pairs,lang_model= lang_model)
plt.figure(1,figsize=(15,15))
wordpairname='bright_vs_dark'
counts=[]
for iplt,wordpairname in enumerate(df_pairs.columns.tolist()):
    plt.subplot(3,3,iplt+1)
    countlist = create_threshold_plot(wordpairname,w2wpe)
    counts.append(countlist)
    plt.plot(thlist,countlist,marker='.')
    plt.xlabel('threshol');
    plt.ylabel('Number of Words with\nproximity < threshold')
    plt.title(wordpairname)
plt.suptitle(f'{lang_model},Nr unique words: {len(unique_word_list)}')
plt.savefig(f'thresholdplots_{lang_model}.pdf')

In [200]:
counts

[[0,
  3,
  4,
  4,
  4,
  5,
  6,
  8,
  10,
  13,
  14,
  15,
  15,
  18,
  24,
  29,
  41,
  72,
  129,
  232,
  438,
  670],
 [0,
  4,
  5,
  8,
  11,
  17,
  28,
  34,
  49,
  104,
  139,
  161,
  190,
  222,
  267,
  307,
  366,
  428,
  483,
  544,
  605,
  670],
 [0,
  4,
  6,
  6,
  6,
  6,
  8,
  8,
  11,
  15,
  22,
  29,
  39,
  52,
  85,
  179,
  242,
  313,
  381,
  470,
  566,
  670],
 [0,
  5,
  9,
  10,
  12,
  13,
  14,
  21,
  25,
  28,
  32,
  38,
  45,
  65,
  94,
  130,
  197,
  276,
  343,
  452,
  588,
  670],
 [0,
  5,
  9,
  14,
  18,
  25,
  38,
  58,
  81,
  97,
  126,
  196,
  227,
  274,
  311,
  355,
  408,
  451,
  521,
  566,
  621,
  670],
 [0, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 7, 7, 9, 10, 10, 18, 30, 83, 196, 447, 670],
 [0,
  4,
  11,
  11,
  13,
  13,
  16,
  16,
  17,
  18,
  32,
  33,
  38,
  48,
  63,
  96,
  141,
  213,
  291,
  401,
  553,
  670],
 [0,
  4,
  5,
  5,
  5,
  5,
  5,
  5,
  7,
  8,
  8,
  13,
  21,
  33,
  51,
  82,
  134,
  211,
  

In [127]:
def f():
    ipdb.set_trace()
    return wordpair_matcher_dict['bright_vs_dark'].match_word_to_words('sunny')
f()

> <ipython-input-127-c926b92ab5e0>(3)f()
      2     ipdb.set_trace()
----> 3     return wordpair_matcher_dict['bright_vs_dark'].match_word_to_words('sunny')
      4 f()



ipdb>  c


'Bright'

In [76]:
y = np.array([np.where(row['bright_vs_dark']>=50,1,0) for row in rowlist])
y.shape,yhat1.shape

((2667,), (2667,))

In [79]:
yhat_binary = np.array([0 if yhatelem==target_word_pair[0] else 1 for yhatelem in yhat1])
yhat_binary.shape

(2667,)

In [ ]:
len(yhat),len(rowlist)

In [ ]:
accuracy_score(y,yhat_binary)

In [ ]:
yhat1

In [ ]:
df_detailed = pd.DataFrame(index=wordlist)
df_detailed.head(7)

In [ ]:
wordlist = [w for r,w in generate_training_examples(df)]
rowlist =  [r for r,w in generate_training_examples(df)]

acc_scores=dict()
for target_word_pair,opposite_quality_pair in zip(target_word_pairs,colnames):
    y = np.array([np.where(row[opposite_quality_pair]>=50,1,0) for row in rowlist])
    
    print(target_word_pair,opposite_quality_pair)
    w2wm = word_to_words_matcher()
    w2wm.build(target_word_pair)
    yhat1 = np.array(f(wordlist,w2wm,variant=1))
    df_detailed[opposite_quality_pair] = yhat1
    yhat_binary = np.array([0 if yhatelem==target_word_pair[0] else 1 for yhatelem in yhat1])
    acc_score = accuracy_score(y,yhat_binary)
    print(f'{acc_score:1.3f}')
    acc_scores[opposite_quality_pair] = acc_score

In [ ]:
print(df_detailed.shape)
df_detailed.to_excel('predicted_qualities.xlsx')
df_detailed.head(20)

In [ ]:
pd.Series(acc_scores).plot.bar(ylabel='accuracy')
plt.plot(plt.xlim(),[0.5,0.5],'--',c='k')
plt.title(f'Accuracy of Spacy word vectors in predicting\ntext_to_qualities.csv ({len(wordlist)} qualities)')
plt.ylim(0,1)